In [2]:
from requests import get
from bs4 import BeautifulSoup
import bs4
import os
import pandas as pd
import numpy as np
os.chdir('D:\APDS\Project\Yelp\py workspace')

In [3]:
df_business = pd.read_csv('..\\dataset\\RestaurantData\\business.csv')

In [4]:
df_business.head()

,Unnamed: 0,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",NaN,"""581 Howe Ave""",Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...
1,5,o9eMRCWt5PkpLDE0gOPtcQ,"""Messina""",NaN,"""Richterstr. 11""",Stuttgart,BW,70567,48.727200,9.147950,4.0,5,1,Italian;Restaurants
2,10,XOSRcvtaKc_Q5H1SAzN20A,"""East Coast Coffee""",NaN,"""737 West Pike St""",Houston,PA,15342,40.241548,-80.212815,4.5,3,0,Breakfast & Brunch;Gluten-Free;Coffee & Tea;Fo...
3,14,fNMVV_ZX7CJSDWQGdOM8Nw,"""Showmars Government Center""",Uptown,"""600 E 4th St""",Charlotte,NC,28202,35.221647,-80.839345,3.5,7,1,Restaurants;American (Traditional)
4,15,l09JfMeQ6ynYs5MCJtrcmQ,"""Alize Catering""",Yonge and Eglinton,"""2459 Yonge St""",Toronto,ON,M4P 2H6,43.711399,-79.399339,3.0,12,0,Italian;French;Restaurants


In [5]:
df_business.city.value_counts()

Toronto                         7148
Las Vegas                       5902
Phoenix                         3652
Montréal                        3332
Charlotte                       2461
Pittsburgh                      2208
Edinburgh                       1502
Scottsdale                      1451
Cleveland                       1364
Mississauga                     1304
Mesa                            1120
Stuttgart                       1050
Madison                         1026
Tempe                            962
Chandler                         819
Henderson                        816
Markham                          760
Glendale                         666
Gilbert                          524
Scarborough                      516
Brampton                         486
Richmond Hill                    454
North York                       425
Vaughan                          411
Champaign                        392
Peoria                           342
North Las Vegas                  325
O

In [6]:
def crawlRestaurants(html_soup):
    rest_names=[]
    rest_urls=[]
    rest_containers = html_soup.find_all('li')
    i=0
    while (i<len(rest_containers)):
        if(isinstance(rest_containers[i].h3, bs4.element.Tag)):
            rest_names.append(rest_containers[i].h3.a.text)
            rest_urls.append(rest_containers[i].h3.a.get('href'))
        i+=1;
    return rest_names,rest_urls;

In [ ]:
url = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Toronto&start={0}'
rest_names=[]
rest_urls=[]
i=0
while(i<60):
    next_url = url.format(i)
    response = get(next_url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    rest_names,rest_urls = crawlRestaurants(html_soup)
    i+=30

In [20]:
for i in range(len(rest_urls)):
    if '/biz/' not in rest_urls[i]:
        del rest_urls[i]
        del rest_names[i]

In [71]:
len(rest_urls)

0

In [44]:
def crawlForRestReviews(url, review_count):
    review_ids = []
    user_ids = []
    reviews = []
    page_size = 20
    max_page_length = int(review_count) - int(review_count)%int(page_size)
    
    i=0
    while (i<=max_page_length):
        next_url = url.format(i)
        response = get(next_url)
        html_soup = BeautifulSoup(response.text, 'html.parser')
        reviews_container = html_soup.find_all('div', class_ = 'review-list')
        
        for x in range(len(reviews_container)):
            reviews_list = reviews_container[x].find_all('li')
            j=0
            while (j<len(reviews_list)):
                if(isinstance(reviews_list[j].p, bs4.element.Tag)):
                    review_ids.append(reviews_list[j].find('div', class_ = 'review')['data-review-id'])
                    user_ids.append(reviews_list[j].find('div', class_ = 'review')['data-signup-object'].replace('user_id:',''))
                    reviews.append(reviews_list[j].p.text)
                j+=1
        
        i+=page_size    
    
    return review_ids, user_ids, reviews

In [54]:
def crawlForNotRecommendedRestReviews(url, review_count):
    review_ids = []
    user_ids = []
    reviews = []
    page_size = 10
    max_page_length = int(review_count) - int(review_count)%int(page_size)
    
    i=0
    while (i<=max_page_length):
        next_url = url.format(i)
        response = get(next_url)
        html_soup = BeautifulSoup(response.text, 'html.parser')
        reviews_container = html_soup.find_all('div', class_ = 'not-recommended-reviews')
        
        for x in range(len(reviews_container)):
            reviews_list = reviews_container[x].find_all('li')
            j=0
            while (j<len(reviews_list)):
                if(isinstance(reviews_list[j].p, bs4.element.Tag)):                    
                    review_ids.append(reviews_list[j].find('div', class_ = 'review')['data-review-id'])
                    user_ids.append(reviews_list[j].find('span', class_ = 'user-display-name')['data-hovercard-id'])
                    reviews.append(reviews_list[j].p.text)
                j+=1
        
        i+=page_size    
    
    return review_ids, user_ids, reviews

In [62]:
def addDataToCsv(review_ids, user_ids, reviews, filename):
    d = {'Review_Id':review_ids,'User_Id':user_ids, 'Review':reviews}
    df = pd.DataFrame(d)
    with open(filename, 'a') as f:
        df.to_csv(f, header=False)

In [61]:
RECOMMENDED_REIVEWS_FILE = '..\\dataset\\RestaurantData\\recommended.csv'
NOT_RECOMMENDED_REIVEWS_FILE = '..\\dataset\\RestaurantData\\not_recommended.csv'

In [69]:
import csv
 
with open(RECOMMENDED_REIVEWS_FILE, 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['SNo','Review_Id', 'User_Id', 'Review_Text'])

with open(NOT_RECOMMENDED_REIVEWS_FILE, 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['SNo','Review_Id', 'User_Id', 'Review_Text'])


In [ ]:
url = 'https://www.yelp.com'
review_ids = []
user_ids = []
reviews = []
review_count = 0
for rurl in rest_urls:
    next_url = url+rurl+'&start={0}'
    response = get(next_url.format(0))
    html_soup = BeautifulSoup(response.text, 'html.parser')
    biz_container = html_soup.find_all('div', class_ = 'biz-rating')
    rating = biz_container[0].find_all('div', class_ = 'i-stars')[0]['title'].strip().replace(' star rating','')
    review_count = biz_container[0].find_all('span')[0].text.strip().replace(' reviews','')
    review_ids, user_ids, reviews = crawlForRestReviews(next_url, review_count)
    addDataToCsv(review_ids, user_ids, reviews, RECOMMENDED_REIVEWS_FILE)

In [66]:
url = 'https://www.yelp.com'
review_ids = []
user_ids = []
reviews = []
review_count = 0
for rurl in rest_urls:
    next_url = url+rurl.replace('biz','not_recommended_reviews')+'&not_recommended_start={0}'
    response = get(next_url.format(0))
    html_soup = BeautifulSoup(response.text, 'html.parser')
    container = html_soup.find_all('div', class_ = 'not-recommended-reviews')
    review_count = container[0].find('h3').text.strip()
    review_count = review_count[:review_count.index(' ')]
    #print(review_count) 
    review_ids, user_ids, reviews = crawlForNotRecommendedRestReviews(next_url, review_count)
    addDataToCsv(review_ids, user_ids, reviews, NOT_RECOMMENDED_REIVEWS_FILE)